In [1]:
!pip install -q transformers rich torchmetrics sklearn pytorch-lightning sentence-transformers

In [2]:
import pandas as pd
import numpy as np
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import dask.bag as db
import json
import zipfile
from sklearn.preprocessing import MultiLabelBinarizer
import csv
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
from sklearn.model_selection import train_test_split
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import RichProgressBar
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from torchmetrics import F1
from torchmetrics.functional import accuracy
from transformers import RobertaTokenizer, RobertaModel, AutoTokenizer, AutoModel
from transformers import AutoModel, AdamW, get_cosine_schedule_with_warmup
import math
from torchmetrics.functional.classification import auroc
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
import pickle



In [3]:
!mkdir dataset

In [4]:
raw_json_data = "arxiv-metadata-oai-snapshot-json.zip"
pickled_dataset = 'dataset_pickle.pkl'
pickled_dataset_onehot = 'dataset_multilabel.pkl'

In [5]:
def __iter__(self): return 0

client_c2b4ce19d76d4b7c87bdf6c8c84d662c = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='SECRET',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

streaming_body_1 = client_c2b4ce19d76d4b7c87bdf6c8c84d662c.get_object(Bucket='SECRET', Key='arxiv-metadata-oai-snapshot.json.zip')['Body']

def save_to_cloud(key, file):
    client_c2b4ce19d76d4b7c87bdf6c8c84d662c.upload_file(Bucket='SECRET', Key=f'snapshot-{file}', Filename=file)

    
def get_file(key, filename):
    client_c2b4ce19d76d4b7c87bdf6c8c84d662c.download_file(Bucket='SECRET', Key=key, Filename=filename)

In [6]:
try:
    get_file('arxiv-metadata-oai-snapshot.pkl', 'dataset_pickle.pkl')
except:
    print("Exception while pulling, will recreate from scratch")
    

if not os.path.exists(pickled_dataset):
    get_file('arxiv-metadata-oai-snapshot.json.zip', raw_json_data)
    with zipfile.ZipFile(raw_json_data, 'r') as zip_ref:
        zip_ref.extractall("dataset")
    docs = db.read_text('./dataset/arxiv-metadata-oai-snapshot.json').map(json.loads)
    trim = lambda x: {'id': x['id'],
                  'category':x['categories'].split(' '),
                  'abstract':x['abstract'].replace('\t', ' ')}
    docs_df = (docs.map(trim).compute())
    df = pd.DataFrame(docs_df)
    df.to_pickle(pickled_dataset)
    save_to_cloud('arxiv-metadata-oai-snapshot.pkl', pickled_dataset)    

In [7]:
df = pd.read_pickle('dataset_pickle.pkl')

In [8]:
df

,id,category,abstract
0,0704.0001,[hep-ph],A fully differential calculation in perturba...
1,0704.0002,"[math.CO, cs.CG]","We describe a new algorithm, the $(k,\ell)$-..."
2,0704.0003,[physics.gen-ph],The evolution of Earth-Moon system is descri...
3,0704.0004,[math.CO],We show that a determinant of Stirling cycle...
4,0704.0005,"[math.CA, math.FA]",In this paper we show how to compute the $\L...
...,...,...,...
2068275,supr-con/9608008,"[supr-con, cond-mat.supr-con]",We report on measurements of the angular dep...
2068276,supr-con/9609001,"[supr-con, cond-mat.supr-con]",The non-linear microwave surface impedance o...
2068277,supr-con/9609002,"[supr-con, cond-mat.supr-con]",The vortex contribution to the dc field (H) ...
2068278,supr-con/9609003,"[supr-con, cond-mat.supr-con]",We show that the density of states in an ani...


In [9]:
# try:
#     get_file('snapshot-dataset_multilabel.pkl', 'dataset_multilabel.pkl')
#     print("Got S3 file")
# except:
#     print("Exception while pulling, will recreate from scratch")
    
# if not os.path.exists('dataset_multilabel.pkl'):
#     mlb = MultiLabelBinarizer()
#     labels = mlb.fit_transform(df.category)
#     df_onehot = pd.concat([df[['abstract']], pd.DataFrame(labels)], axis=1)
#     df_onehot.columns = ['abstract'] + list(mlb.classes_)
#     df_onehot.to_pickle('dataset_multilabel.pkl')
#     save_to_cloud('snapshot-dataset_multilabel.pkl', 'dataset_multilabel.pkl')

In [10]:
# df_onehot = pd.read_pickle('dataset_multilabel.pkl')

In [68]:
# LABEL_COLUMNS = df_onehot.columns.tolist()[1:]
# LABEL_COLUMNS

## Trying preprocessing?

In [12]:
get_file('arxiv-metadata-oai-snapshot.json.zip', raw_json_data)
with zipfile.ZipFile(raw_json_data, 'r') as zip_ref:
    zip_ref.extractall("dataset")
docs = db.read_text('./dataset/arxiv-metadata-oai-snapshot.json').map(json.loads)


In [13]:
get_latest_version = lambda x: x['versions'][-1]['created']
trim = lambda x: {'id': x['id'],
              'category':x['categories'].split(' '),
              'abstract':x['abstract'].replace('\t', ' ')}
docs_df = (docs.filter(lambda x: int(get_latest_version(x).split(' ')[3]) > 2009).map(trim).compute())
df = pd.DataFrame(docs_df)
df.to_pickle("df2009.pkl")
save_to_cloud('df2009.pkl', 'df2009.pkl')

In [6]:
get_file('snapshot-df2009.pkl', 'df2009.pkl')
df = pd.read_pickle('df2009.pkl')
base_df = pd.read_pickle('df2009.pkl')

In [7]:
%%time
df["abstract"] = df["abstract"].apply(lambda text: " ".join([word for word in text.replace("\n", " ").replace("\t", " ").strip().split(" ") if len(word) > 0]))

CPU times: user 27.4 s, sys: 653 ms, total: 28.1 s
Wall time: 28 s


In [8]:
frequent_categories = pd.Series(np.concatenate(df['category'])).value_counts().reset_index(name="count").query("count > 1000")["index"].tolist()

In [9]:
df = df[df["category"].apply(lambda cats: all(elem in frequent_categories for elem in cats))]

In [10]:
df = df[df["abstract"].apply(lambda text: len(text.split()) > 128)]

In [11]:
df = df.reset_index()

In [12]:
categories_filtered = pd.Series(np.concatenate(df['category'])).value_counts()

In [13]:
from rich.progress import track

# Select 5k randomly, then at least 1k from each category, then drop duplicates
uni_df = df.sample(n = 5000)
for category, _ in track(categories_filtered.items(), total=len(categories_filtered), description="Pulling 1k from each cat"):
    subset_df = df[df["category"].apply(lambda cats: category in cats)]
    uni_df = pd.concat([subset_df.sample(n = min(5000, len(subset_df))), uni_df], ignore_index=True, sort=False)

Pulling 1k from each cat ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00

In [14]:
uni_df = uni_df.iloc[uni_df.astype(str).drop_duplicates().index]


In [15]:
uni_df

,index,id,category,abstract
0,1452332,2202.02938,[math.GM],The defining property of an artificial physica...
1,441387,1501.04069,[math.GM],In this paper the certain 4-dimensional algebr...
2,185025,1204.2193,[math.GM],An alternative mathematics based on qualitativ...
3,713686,1706.08381,[math.GM],"In any cubic polynomial, the average of the sl..."
4,1408117,2111.03444,[math.GM],"For the first time, a general fractional calcu..."
...,...,...,...,...
553441,899287,1811.04084,[hep-ph],Precision measurements of high energy top quar...
553445,621796,1609.05748,[astro-ph.IM],Wide-field (> 100 deg$^2$) hard X-ray coded-ap...
553447,1148273,2006.03189,"[cs.CL, cs.LG]",Automatic evaluation of various text quality c...
553451,857348,1807.09570,[cond-mat.mtrl-sci],Ferrimagnetic TbFe or TbFeCo amorphous alloy t...


In [16]:
base_df_without_subset = base_df[~base_df['id'].isin(uni_df['id'])]


In [17]:
base_df_without_subset

,id,category,abstract
1,0704.0038,"[physics.optics, physics.comp-ph]",We present a review of the discrete dipole a...
2,0704.0073,[math.AG],We treat Koll\'ar's injectivity theorem from...
4,0704.0142,[cond-mat.stat-mech],A nonperturbative renormalization of the phi...
6,0704.0213,[cs.CC],This article has been withdrawn because it h...
7,0704.0304,"[cs.IT, cs.AI, math.IT, q-bio.PE]",This paper discusses the benefits of describ...
...,...,...,...
1507137,quant-ph/9911015,"[quant-ph, nucl-th]",Only craziness I am ashamed about.\n
1507138,quant-ph/9911085,"[quant-ph, nucl-th]",Only craziness I am ashamed about.\n
1507139,quant-ph/9911122,"[quant-ph, cond-mat.dis-nn, hep-th]",Only craziness I am ashamed about.\n
1507140,solv-int/9909027,"[solv-int, math-ph, math.AP, math.MP, nlin.SI]",We use the Calogero equation to illustrate t...


In [18]:
base_df_categories = pd.Series(np.concatenate(base_df['category'])).value_counts().reset_index(name="count")

In [19]:
all_categories = base_df_categories["index"].tolist()

In [20]:
mlb = MultiLabelBinarizer(classes=all_categories)
labels = mlb.fit_transform(df.category)

In [21]:
uni_df = uni_df.reset_index()

In [22]:

labels = mlb.fit_transform(uni_df.category)
df_onehot = pd.concat([uni_df[['abstract']], pd.DataFrame(labels)], axis=1)
df_onehot.columns = ['abstract'] + list(mlb.classes_)


In [23]:
df_onehot.to_pickle('cleaned_data_for_training.pkl')
save_to_cloud('cleaned_data_for_training.pkl', 'cleaned_data_for_training.pkl')

In [24]:
base_df_without_subset = base_df_without_subset.reset_index()

In [25]:
labels = mlb.fit_transform(base_df_without_subset.category)
df_onehot_base_df_without_subset = pd.concat([base_df_without_subset[['abstract']], pd.DataFrame(labels)], axis=1)
df_onehot_base_df_without_subset.columns = ['abstract'] + list(mlb.classes_)

In [26]:
df_rest, df_onehot_base_df_without_subset_test = train_test_split(df_onehot_base_df_without_subset, test_size=25000, random_state=1337, shuffle=True)
_, df_onehot_base_df_without_subset_valid   = train_test_split(df_rest, test_size=25000, random_state=1337, shuffle=True)

In [27]:
df_onehot_base_df_without_subset_test = df_onehot_base_df_without_subset_test.reset_index()

In [28]:
df_onehot_base_df_without_subset_valid = df_onehot_base_df_without_subset_valid.reset_index()

In [29]:
df_onehot_base_df_without_subset_test.to_pickle('df_onehot_base_df_without_subset_test.pkl')
save_to_cloud('df_onehot_base_df_without_subset_test.pkl', 'df_onehot_base_df_without_subset_test.pkl')

In [30]:
df_onehot_base_df_without_subset_valid.to_pickle('df_onehot_base_df_without_subset_valid.pkl')
save_to_cloud('df_onehot_base_df_without_subset_valid.pkl', 'df_onehot_base_df_without_subset_valid.pkl')

In [31]:
df_onehot_base_df_without_subset_valid

,index,abstract,cs.LG,quant-ph,hep-ph,hep-th,cs.CV,gr-qc,cond-mat.mes-hall,cond-mat.mtrl-sci,...,cond-mat,alg-geom,q-alg,q-bio,dg-ga,adap-org,chao-dyn,funct-an,solv-int,patt-sol
0,166891,We use data from the WMAP temperature maps t...,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,913442,We update our previous results for (pseudo-)...,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,800337,We present a class of Arbitrary Lagrangian-E...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,546814,We propose a realization of a synthetic Rand...,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,934545,Granger causal modeling is an emerging topic...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,973130,We present the results of semi-inclusive neu...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24996,992303,Harnessing the recent advance in data scienc...,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
24997,1018098,Hematite at room temperature is a weak ferro...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
24998,69256,We argue that the eta-problem in supergravit...,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


## Model

In [11]:
get_file('snapshot-cleaned_data_for_training.pkl', 'cleaned_data_for_training.pkl')
get_file('snapshot-df_onehot_base_df_without_subset_test.pkl', 'df_onehot_base_df_without_subset_test.pkl')
get_file('snapshot-df_onehot_base_df_without_subset_valid.pkl', 'df_onehot_base_df_without_subset_valid.pkl')

In [12]:
df_onehot = pd.read_pickle('cleaned_data_for_training.pkl')
df_rest_test = pd.read_pickle('df_onehot_base_df_without_subset_test.pkl')
df_rest_valid = pd.read_pickle('df_onehot_base_df_without_subset_valid.pkl')

In [7]:
# df_onehot = pd.read_pickle('uni_df.pkl')
# LABEL_COLUMNS = df_onehot.columns.tolist()[1:]

In [8]:
# embeddings_cache = "all-mpnet-base-df.pkl"

In [35]:

# model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
# df_onehot_abstracts = df_onehot["abstract"]

In [36]:
# %%time


# embeddings = model.encode(df_onehot_abstracts, show_progress_bar=True, batch_size=32, device=torch.device('cuda'))

# with open(embeddings_cache, "wb") as fOut:
#     pickle.dump({'sentences': df_onehot_abstracts.tolist(), 'embeddings': embeddings}, fOut, protocol=pickle.HIGHEST_PROTOCOL)

Batches:   0%|          | 0/4287 [00:00<?, ?it/s]

CPU times: user 1h 20min 54s, sys: 24min, total: 1h 44min 54s
Wall time: 1h 40min 52s


In [37]:
# save_to_cloud(embeddings_cache, embeddings_cache)

In [9]:
# get_file(f'snapshot-{embeddings_cache}', embeddings_cache)

In [10]:
# import joblib
# X_all = joblib.load(embeddings_cache)
# X_all = X_all['embeddings']

In [11]:
# df_onehot["embeddings"] = X_all.tolist()

In [12]:
# Baseline?

In [13]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([
    ('vct', TfidfVectorizer(strip_accents='unicode', stop_words='english', max_df=0.9, sublinear_tf=True)),
    ('clf', OneVsRestClassifier(LogisticRegression(C=1.1, multi_class='multinomial', max_iter=300, verbose=1, n_jobs=-1)))
])

In [14]:
RANDOM_SEED = 1337
df_train, df_test = train_test_split(df_onehot, test_size=0.1, random_state=RANDOM_SEED, shuffle=True)
df_train, df_validation = train_test_split(df_train, test_size=0.2, random_state=RANDOM_SEED, shuffle=True)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_validation = df_validation.reset_index(drop=True)
df_rest_test = df_rest_test.reset_index(drop=True)
df_rest_valid = df_rest_valid.reset_index(drop=True)

In [15]:
X_train = df_train["abstract"]
y_train = df_train.loc[:, ~df_train.columns.isin(['abstract', 'embeddings'])]
X_test = df_test["abstract"]
y_test = df_test.loc[:, ~df_test.columns.isin(['abstract', 'embeddings'])]
X_rest_test = df_rest_test["abstract"]
y_rest_test = df_rest_test.loc[:, ~df_rest_test.columns.isin(['index', 'abstract', 'embeddings'])]

In [17]:
%%time
pipeline.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =       531774     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.07950D+05    |proj g|=  1.21654D+05


 This problem is unconstrained.



At iterate   50    f=  4.04440D+04    |proj g|=  3.62537D+01

At iterate  100    f=  3.99162D+04    |proj g|=  5.65372D+00

At iterate  150    f=  3.99102D+04    |proj g|=  1.35573D+00

At iterate  200    f=  3.99101D+04    |proj g|=  6.67480D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
*****    208    242      1     0     0   3.704D-02   3.991D+04
  F =   39910.055849540251     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.5min finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


...

 This problem is unconstrained.



At iterate   50    f=  2.49523D+03    |proj g|=  1.25290D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
*****     55     62      1     0     0   3.265D-02   2.495D+03
  F =   2495.2261376836882     

CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH             


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   42.4s finished
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/multiclass.py:79: UserWarning: Label not 151 is present in all training examples.
  warnings.warn(
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/multiclass.py:79: UserWarning: Label not 152 is present in all training examples.
  warnings.warn(
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/multiclass.py:79: UserWarning: Label not 153 is present in all training examples.
  warnings.warn(
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/multiclass.py:79: UserWarning: Label not 154 is present in all training examples.
  warnings.warn(
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/multiclass.py:79: UserWarning: Label not 155 is present in all training examples.
  warnings.warn(
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/multiclass.py:79: UserWarning: Label not 156 is present 

CPU times: user 1min 26s, sys: 49.5 s, total: 2min 15s
Wall time: 3h 25min 38s


Pipeline(steps=[('vct',
                 TfidfVectorizer(max_df=0.9, stop_words='english',
                                 strip_accents='unicode', sublinear_tf=True)),
                ('clf',
                 OneVsRestClassifier(estimator=LogisticRegression(C=1.1,
                                                                  max_iter=300,
                                                                  multi_class='multinomial',
                                                                  n_jobs=-1,
                                                                  verbose=1)))])

In [18]:
X_test_preds_proba = pipeline.predict_proba(X_test)

In [19]:
def predict(tresh = 0.5, preds_proba = X_test_preds_proba, y_test = y_test):
    y_pred = [[int(val >= tresh) for val in pred] for pred in preds_proba]
    y_true = y_test
    print(classification_report(y_true, y_pred))
    print("Acc score: ", accuracy_score(y_true, y_pred))

In [20]:
predict()

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.70      0.55      0.61      3988
           1       0.80      0.62      0.70      1904
           2       0.84      0.68      0.75      1552
           3       0.80      0.59      0.68      1492
           4       0.82      0.66      0.73      1841
           5       0.80      0.62      0.70      1163
           6       0.70      0.49      0.57      1172
           7       0.68      0.48      0.56      1264
           8       0.79      0.57      0.66      1018
           9       0.59      0.27      0.37      1818
          10       0.68      0.31      0.42      1451
          11       0.68      0.31      0.42      1451
          12       0.76      0.66      0.71       883
          13       0.79      0.65      0.72      1136
          14       0.78      0.54      0.64       974
          15       0.60      0.21      0.32      1728
          16       0.79      0.55      0.65       727
          17       0.84    

Acc score:  0.24519426912092923


In [27]:
predict(0.22)

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.57      0.78      0.66      3988
           1       0.66      0.77      0.71      1904
           2       0.72      0.84      0.78      1552
           3       0.66      0.78      0.72      1492
           4       0.70      0.81      0.75      1841
           5       0.73      0.79      0.76      1163
           6       0.57      0.70      0.63      1172
           7       0.56      0.71      0.62      1264
           8       0.64      0.75      0.69      1018
           9       0.42      0.58      0.49      1818
          10       0.51      0.60      0.55      1451
          11       0.51      0.60      0.55      1451
          12       0.64      0.82      0.72       883
          13       0.67      0.82      0.74      1136
          14       0.59      0.73      0.65       974
          15       0.41      0.48      0.44      1728
          16       0.61      0.74      0.67       727
          17       0.71    

Acc score:  0.21260409417072643


In [22]:
predict(0.15)

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.52      0.86      0.65      3988
           1       0.60      0.82      0.69      1904
           2       0.66      0.88      0.75      1552
           3       0.59      0.83      0.69      1492
           4       0.65      0.85      0.74      1841
           5       0.69      0.85      0.76      1163
           6       0.50      0.77      0.61      1172
           7       0.50      0.78      0.61      1264
           8       0.59      0.81      0.68      1018
           9       0.37      0.68      0.48      1818
          10       0.43      0.69      0.53      1451
          11       0.43      0.69      0.53      1451
          12       0.59      0.86      0.70       883
          13       0.63      0.87      0.73      1136
          14       0.53      0.81      0.64       974
          15       0.35      0.60      0.44      1728
          16       0.56      0.80      0.66       727
          17       0.64    

Acc score:  0.16875854952122682


In [23]:
X_rest_test_preds_proba = pipeline.predict_proba(X_rest_test)

In [25]:
predict(0.2, X_rest_test_preds_proba, y_rest_test)

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.56      0.86      0.67      1563
           1       0.73      0.83      0.78      1609
           2       0.74      0.83      0.78      1636
           3       0.66      0.80      0.72      1504
           4       0.84      0.86      0.85      1271
           5       0.68      0.80      0.74      1113
           6       0.62      0.77      0.69      1065
           7       0.58      0.77      0.66      1008
           8       0.60      0.78      0.68      1081
           9       0.41      0.62      0.50       796
          10       0.32      0.51      0.39       851
          11       0.32      0.51      0.39       851
          12       0.60      0.87      0.71       937
          13       0.64      0.86      0.74       858
          14       0.68      0.75      0.71       821
          15       0.30      0.52      0.38       608
          16       0.72      0.75      0.74       855
          17       0.66    

Acc score:  0.25576


In [114]:
# from sklearn.metrics import confusion_matrix
# y_pred = [[int(val >= 0.2) for val in pred] for pred in preds_proba]
# y_true = y_test
# cm = multilabel_confusion_matrix(y_true, y_pred)

AttributeError: 'DataFrame' object has no attribute 'argmax'

In [57]:
# df_onehot_subs = df_onehot

In [58]:
# df_onehot_subs

,abstract,astro-ph.CO,astro-ph.EP,astro-ph.GA,astro-ph.HE,astro-ph.IM,astro-ph.SR,cond-mat.dis-nn,cond-mat.mes-hall,cond-mat.mtrl-sci,...,q-fin.RM,q-fin.ST,q-fin.TR,quant-ph,stat.AP,stat.CO,stat.ME,stat.ML,stat.OT,stat.TH
0,Exact solutions with the initial conditions ar...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,In this paper we develop three characterizatio...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,In this article we continue a previous work in...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The fundamental theorem of affine geometry is ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The subject matter of this work is the set of ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137164,Fast and accurate solution of time-dependent p...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
137165,We study latent heat and the pressure gap betw...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
137166,We perform a rigorous analysis of the quasineu...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
137167,A Symplectic Effective Field Theory that unvei...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
class ArxivAbstracts(Dataset):
    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.text = df.abstract
        self.labels = df[df.columns.difference(['abstract'])]
        self.max_len = max_len
        
    def __len__(self):
        return len(self.text)
    
    def __getitem__(self, item_idx):
        text = self.text[item_idx]
        encoding = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            return_token_type_ids=False,
            return_attention_mask= True,
            truncation=True,
            return_tensors='pt'
          )
               
        return {
          'ids': encoding["input_ids"].flatten(),
          'mask': encoding["attention_mask"].flatten(),
          'targets': torch.tensor(self.labels.iloc[item_idx].tolist(), dtype=torch.float)  
        }

In [93]:
RANDOM_SEED = 1337
df_train, df_test = train_test_split(df_onehot, test_size=0.1, random_state=RANDOM_SEED, shuffle=True)
df_train, df_validation = train_test_split(df_train, test_size=0.2, random_state=RANDOM_SEED, shuffle=True)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_validation = df_validation.reset_index(drop=True)

In [94]:
import pytorch_lightning as pl
class ArxivAbstractsModule(pl.LightningDataModule):
    
    def __init__(self, df_train, df_test, df_validation, tokenizer, batch_size=16, max_token_len=256):
        super().__init__()
        self.df_train = df_train
        self.df_test = df_test
        self.df_validation = df_validation
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.max_token_len = max_token_len

    def setup(self, stage=None):
        self.train_dataset = ArxivAbstracts(df=self.df_train, tokenizer=self.tokenizer, max_len= self.max_token_len)
        self.val_dataset = ArxivAbstracts(df=self.df_validation, tokenizer=self.tokenizer, max_len= self.max_token_len)
        self.test_dataset = ArxivAbstracts(df=self.df_test, tokenizer=self.tokenizer, max_len= self.max_token_len)


    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)


In [95]:
tokenizer = AutoTokenizer.from_pretrained('distilroberta-base')
arxiv_data = ArxivAbstractsModule(df_train, df_test, df_validation, tokenizer, max_token_len=384)
arxiv_data.setup()

In [96]:
class ArxivAbstractsClassifier(pl.LightningModule):
    def __init__(self, config):
        super().__init__()

        self.config = config
        self.pretrained_model = AutoModel.from_pretrained(config['model_name'], return_dict=True)
        self.hidden = torch.nn.Linear(self.pretrained_model.config.hidden_size, self.pretrained_model.config.hidden_size)
        self.classifier = torch.nn.Linear(self.pretrained_model.config.hidden_size, self.config['n_labels']) 
        torch.nn.init.xavier_uniform_(self.classifier.weight)
        self.loss_func = torch.nn.BCEWithLogitsLoss(reduction='mean')
        self.dropout = torch.nn.Dropout()
        self.save_hyperparameters()
    
    def forward(self, ids, mask, targets=None):
        # roberta layer
        output = self.pretrained_model(input_ids=ids, attention_mask=mask)
        pooled_output = torch.mean(output.last_hidden_state, 1)
        # final logits
        pooled_output = self.dropout(pooled_output)
        pooled_output = self.hidden(pooled_output)
        pooled_output = F.relu(pooled_output)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        # calculate loss
        loss = 0
        if targets is not None:
            loss = self.loss_func(logits.view(-1, self.config['n_labels']), targets.view(-1, self.config['n_labels']))
        return loss, logits
  
    def training_step(self, batch, batch_index):
        loss, outputs = self(**batch)
        self.log("train_loss", loss)
        return {"loss":loss, "predictions": outputs, "labels": batch["targets"]}

    def validation_step(self, batch, batch_index):
        loss, outputs = self(**batch)
        self.log("val_loss", loss)
        return {"val_loss": loss, "predictions": outputs, "labels": batch["targets"]}

    def predict_step(self, batch, batch_index):
        loss, outputs = self(**batch)
        return torch.sigmoid(outputs)

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=self.config['lr'], weight_decay=self.config['weight_decay'])
        total_steps = self.config['train_size']/self.config['batch_size']
        warmup_steps = math.floor(total_steps * self.config['warmup'])
        warmup_steps = math.floor(total_steps * self.config['warmup'])
        scheduler = get_cosine_schedule_with_warmup(optimizer, warmup_steps, total_steps)
        return [optimizer],[scheduler]

In [97]:
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    monitor='val_loss',
    filename='ArxivAbstractsClassifierPreprocess-{epoch:02d}-{val_loss:.2f}',
    save_top_k=2,
    verbose=True,
    mode='min'
)
logger = TensorBoardLogger("lightning_logs", name="arxiv-abstracts-logs")

config = {
    'model_name': 'distilroberta-base',
    'n_labels': len(LABEL_COLUMNS),
    'batch_size': 128,
    'lr': 1.5e-6,
    'warmup': 0.2, 
    'train_size': len(arxiv_data.train_dataloader()),
    'weight_decay': 0.001,
    'n_epochs': 10
}


In [98]:
%%time 
model = ArxivAbstractsClassifier(config)
trainer = Trainer(max_epochs = config['n_epochs'], 
                  gpus = 1, 
                  logger=logger, 
                  enable_checkpointing=True, 
                  callbacks=[checkpoint_callback, RichProgressBar()])


trainer.fit(model, datamodule=arxiv_data)

Epoch 0    ━━━━━━━━━━━━━━━━━━━━━━━━━ 47/7717 0:00:48 • 2:09:52 0.98it/s loss: 0.708 v_num: 5 

/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer
.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")

CPU times: user 42 s, sys: 11.7 s, total: 53.7 s
Wall time: 50.8 s


In [81]:
model_name = "ArxivAbstractsClassifierPreprocessData5EpochRoBERTa"
def save_dir_to_cloud(path, model_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                filepath = os.path.join(root,file)
                save_to_cloud(f"outs/{model_name}/{file}", filepath)

In [82]:
trainer.save_checkpoint(f"{model_name}.ckpt")
save_to_cloud(f'{model_name}.ckpt', f"{model_name}.ckpt")

In [83]:
save_dir_to_cloud("lightning_logs", model_name)
save_dir_to_cloud("checkpoints", model_name)

In [84]:
get_file(f'snapshot-{model_name}.ckpt', f"snapshot-{model_name}.ckpt")
# trainer = Trainer()
# trainer.fit(model, datamodule=arxiv_data, ckpt_path=f"snapshot-{model_name}.ckpt")

In [87]:

model = ArxivAbstractsClassifier(config).load_from_checkpoint(f"snapshot-{model_name}.ckpt", config=config)
trainer = Trainer(gpus = 1, 
                  logger=logger,
                  callbacks=[RichProgressBar()])

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializin

In [ ]:
# trainer.test(model, datamodule=arxiv_data)

In [88]:
preds = trainer.predict(model, dataloaders=arxiv_data.test_dataloader())

Predicting ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7/7 0:00:02 • 0:00:00 2.49it/s

In [89]:
preds = np.concatenate(preds, axis=0)

In [90]:
# Probably needs a more sophisticated thingy with model.eval + freezing
# Or just incorporating that to the model above so that loading from a checkpoint will work correctly
def predict(tresh = 0.5):
    y_pred = [[int(val >= tresh) for val in pred] for pred in preds]
    y_true = [tensor["targets"] for tensor in arxiv_data.test_dataloader()]
    y_true = np.concatenate([tensor.tolist() for tensor in y_true])
    print(classification_report(y_true, y_pred))
    print("Acc score: ", accuracy_score(y_true, y_pred))

In [91]:
predict(tresh = 0.35)

              precision    recall  f1-score   support

           0       0.02      1.00      0.04         2
           1       0.01      1.00      0.02         1
           2       0.01      1.00      0.02         1
           3       0.02      1.00      0.04         2
           4       0.02      1.00      0.04         2
           5       0.02      1.00      0.04         2
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       0.01      1.00      0.02         1
          12       0.02      1.00      0.04         2
          13       0.02      1.00      0.04         2
          14       0.02      1.00      0.04         2
          15       0.04      1.00      0.08         4
          16       0.02      1.00      0.04         2
          17       0.02    

/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [92]:
predict(tresh = 0.4)

              precision    recall  f1-score   support

           0       0.02      1.00      0.04         2
           1       0.01      1.00      0.02         1
           2       0.01      1.00      0.02         1
           3       0.02      1.00      0.04         2
           4       0.02      1.00      0.04         2
           5       0.02      1.00      0.04         2
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       0.01      1.00      0.02         1
          12       0.02      1.00      0.04         2
          13       0.02      1.00      0.04         2
          14       0.02      1.00      0.04         2
          15       0.04      1.00      0.08         4
          16       0.02      1.00      0.04         2
          17       0.02    

/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/envs/Python-3.9-CUDA/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [5]:
predict(tresh = 0.5)

In [ ]:
predict(tresh = 0.2)